In [9]:
import os
from collections import Counter, defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import json
import csv
import pandas as pd
import seaborn as sns

In [2]:
sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [19]:
ROOT = '/mnt/data0/lucy/manosphere/'
SUBS = ROOT + 'data/submissions/'
SUB_META = ROOT + 'data/subreddits.txt'
LOGS = ROOT + 'logs/'
PEOPLE_FILE = ROOT + 'data/people.csv'
SUB_META = ROOT + 'data/subreddits.txt'

# Glossary matches

In [82]:
def get_manual_people(): 
    """
    get list of words, add plural forms
    copied from find_people.py
    """
    words = set()
    sing2plural = {}
    with open(PEOPLE_FILE, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['community'].strip() != 'generic': 
                word_sing = row['word (singular)'].strip()
                plural = row['word (plural)'].strip()
                if word_sing != '':
                    if word_sing.lower() in words: print('REPEAT', word_sing)
                    words.add(word_sing.lower())
                    sing2plural[word_sing] = plural
                if plural != '': 
                    if plural.lower() in words: print('REPEAT', plural)
                    assert word_sing != ''
                    words.add(plural.lower())
    return words, sing2plural

In [20]:
def get_subreddit_categories(): 
    categories = defaultdict(str)
    categories_rev = defaultdict(list)
    with open(SUB_META, 'r') as infile: 
        reader = csv.DictReader(infile)
        for row in reader: 
            name = row['Subreddit'].strip().lower()
            if name.startswith('/r/'): name = name[3:]
            if name.startswith('r/'): name = name[2:]
            if name.endswith('/'): name = name[:-1]
            categories[name] = row['Category after majority agreement']
            categories_rev[row['Category after majority agreement']].append(name)
    return categories, categories_rev

In [21]:
categories, categories_rev = get_subreddit_categories()

In [4]:
with open(LOGS + 'glossword_one_month.temp', 'r') as infile: 
    d = json.load(infile)

In [29]:
people, sing2plural = get_manual_people()
people_counts_sr = defaultdict(Counter)
people_counts_cat = defaultdict(Counter)
for sr in d: 
    cat = categories[sr[8:]]
    for term in d[sr]: 
        if term in sing2plural:
            plural = sing2plural[term]
            if plural not in d[sr]: 
                plural_count = 0
            else: 
                plural_count = d[sr][plural]
            total_count = d[sr][term] + plural_count
            people_counts_sr[sr][term] = total_count
            people_counts_cat[cat][term] += total_count

In [31]:
for sr in people_counts_sr: 
    print(sr)
    for tup in people_counts_sr[sr].most_common(10): 
        if tup[1] > 0: 
            print(tup,)
    print()

2016-03_depression
('gf', 184)
('natural', 163)
('bro', 96)
('void', 95)
('bf', 90)
('tool', 82)
('primary', 61)
('obstacle', 46)
('robot', 45)
('target', 34)

2016-03_foreveralone
('normie', 280)
('gf', 164)
('bro', 90)
('nice guy', 89)
('natural', 64)
('bf', 60)
('alpha', 60)
('jaw', 48)
('beta', 39)
('pua', 34)

2016-03_mensrights
('mra', 1004)
('sjw', 452)
('wing', 379)
('primary', 197)
('mgtow', 182)
('natural', 144)
('target', 139)
('tool', 138)
('misogynist', 135)
('bro', 102)

2016-03_theredpill
('beta', 2651)
('alpha', 2590)
('natural', 659)
('bro', 446)
('mgtow', 388)
('pua', 382)
('tool', 315)
('sjw', 303)
('gf', 296)
('unicorn', 245)

2016-03_thebluepill
('alpha', 695)
('beta', 284)
('mgtow', 176)
('cuck', 136)
('mra', 133)
('bro', 102)
('natural', 99)
('misogynist', 82)
('nice guy', 75)
('alpha male', 75)

2016-03_asktrp
('beta', 1085)
('alpha', 943)
('gf', 310)
('bro', 309)
('natural', 264)
('orbiter', 170)
('nice guy', 139)
('pua', 116)
('bf', 111)
('tool', 100)

2016-03

In [32]:
for cat in people_counts_cat:
    print(cat)
    for tup in people_counts_cat[cat].most_common(10): 
        if tup[1] > 0: 
            print(tup,)
    print()

Health
('natural', 261)
('gf', 242)
('bf', 142)
('bro', 141)
('void', 132)
('tool', 124)
('primary', 83)
('obstacle', 62)
('robot', 56)
('lover', 50)

Incels
('normie', 378)
('gf', 186)
('incel', 135)
('nice guy', 129)
('alpha', 117)
('bro', 104)
('beta', 79)
('natural', 74)
('bf', 66)
('pua', 63)

MRA
('mra', 1035)
('sjw', 463)
('wing', 382)
('primary', 198)
('mgtow', 182)
('tool', 146)
('natural', 145)
('target', 139)
('misogynist', 136)
('bro', 107)

TRP
('beta', 4159)
('alpha', 4002)
('natural', 1032)
('bro', 788)
('gf', 641)
('pua', 530)
('tool', 507)
('mgtow', 466)
('nice guy', 419)
('sjw', 384)

Criticism
('alpha', 731)
('beta', 318)
('mgtow', 218)
('mra', 187)
('cuck', 138)
('bro', 106)
('natural', 106)
('nice guy', 93)
('misogynist', 88)
('sjw', 81)

MGTOW
('mgtow', 2126)
('beta', 251)
('alpha', 179)
('sjw', 152)
('pua', 151)
('bro', 120)
('unicorn', 103)
('gf', 93)
('white knight', 93)
('natural', 92)

PUA
('nice guy', 214)
('pua', 203)
('bro', 178)
('natural', 165)
('alpha',

# BookNLP NER tagged people

In [88]:
POSTS = ROOT + 'data/submissions/'
COMMENTS = ROOT + 'data/comments/'

In [89]:
all_entities = Counter()
with open(LOGS + 'tagged_people/2016-03', 'r') as infile: 
    for line in infile: 
        entities = line.split('\t')
        for entity in entities: 
            if entity.strip() == '': continue
            person = entity.lower().split()[2:]
            if person[0] == 'a' or person[0] == 'an' or person[0] == 'the': 
                person = person[1:]
            person = ' '.join(person)
            all_entities[person] += 1

In [90]:
all_entities.most_common()

[('women', 33676),
 ('people', 24529),
 ('men', 20482),
 ('someone', 10379),
 ('man', 9416),
 ('woman', 8572),
 ('everyone', 6422),
 ('girl', 5592),
 ('girls', 5435),
 ('anyone', 5084),
 ('friends', 4762),
 ('no one', 3983),
 ('child', 3207),
 ('others', 3191),
 ('guy', 3153),
 ('other people', 2850),
 ('guys', 2699),
 ('kids', 2471),
 ('person', 2440),
 ('feminists', 2382),
 ('my parents', 2319),
 ('family', 2254),
 ('children', 2223),
 ('nobody', 2186),
 ('my friends', 1977),
 ('friend', 1817),
 ('my family', 1769),
 ('someone else', 1748),
 ('some people', 1702),
 ('most people', 1693),
 ('dude', 1637),
 ('my mom', 1579),
 ('', 1502),
 ('this guy', 1449),
 ('my dad', 1397),
 ('kid', 1382),
 ('this girl', 1349),
 ('my wife', 1330),
 ('my friend', 1199),
 ('boys', 1167),
 ('doctor', 1150),
 ('father', 1140),
 ('my mother', 1049),
 ('therapist', 1039),
 ('mother', 1011),
 ('wife', 989),
 ('male', 947),
 ('everyone else', 941),
 ('anyone else', 923),
 ('female', 857),
 ('your wife', 843

In [91]:
people, sing2plural = get_manual_people()

In [92]:
uncounted = set()
for person in people: 
    if person in all_entities: 
        print(person, all_entities[person])
    else: 
        uncounted.add(person)

social butterfly 12
misogynists 53
western women 71
mras 168
lone wolves 2
stay at home mothers 2
lover 35
nymphos 1
roastie 1
afc 7
mc 6
foreign woman 4
hot babes 2
chad 106
bf 14
hb 3
real man 131
stalkers 6
incel 8
primary 4
lovers 25
feminazi 25
feminazis 44
naturals 8
robots 27
dancing monkey 3
amog 12
cheaters 30
femtards 1
nice guy 319
western woman 14
lone wolf 30
landwhales 1
entourage 5
bunny 4
orbiters 22
hater 12
gamers 33
red pill women 3
role model 19
traditionalists 11
mgtows 32
djs 5
prude 9
poster boy 1
providers 38
stalker 28
beta males 52
pug 1
provider 109
beta 309
conformists 3
wizard 15
role models 10
nympho 4
incels 3
red pill woman 8
dj 24
neets 2
observers 6
landwhale 6
pick up masters 1
femtard 2
pickup artist 10
cucks 21
unicorn 32
subhuman 1
prudes 4
aspies 4
mra 182
cheater 54
truecel 2
manlet 5
otaku 1
hugger 1
white knights 98
alphas 234
foreign women 15
normies 43
false prophet 1
pick up master 1
wingwomen 3
alpha 467
gf 26
beta male 29
seducer 4
real me

In [93]:
uncounted

{'ab',
 "ab's",
 'absolute beginner',
 'add girl',
 'add girls',
 'afog',
 'afogs',
 'amogs',
 'androphile',
 'androphiles',
 'apua',
 'apuas',
 'arbitrary target',
 'arbitrary target s',
 'bafc',
 'bafcs',
 'becky',
 'beckys',
 'betabuxx',
 'betabuxxes',
 'betafag',
 'betafags',
 'black bunny',
 'black bunnys',
 'blackcel',
 'blackcels',
 'blue bunny',
 'blue bunnys',
 'bottom feeders',
 'breakup artist',
 'breakup artists',
 'bunnys',
 'casanovas',
 'celebrity pua',
 'celebrity puas',
 'chaddam',
 'chaddams',
 'chadlite',
 'chadlites',
 'chadpreet',
 'chadpreets',
 'chadrone',
 'chadrones',
 'chang',
 'changs',
 'chit chat master',
 'chit chat masters',
 'christian pua',
 'christian puas',
 'club girl',
 'club girls',
 'cock tease',
 'cock teases',
 'comfort whore',
 'comfort whores',
 'cougar',
 'cum dumpsters',
 'cum rag',
 'cum rags',
 'cumdumpster',
 'cumdumpsters',
 'currycel',
 'currycels',
 'cybercel',
 'cybercels',
 'dance floor monkey',
 'dance floor monkeys',
 'dancing monk

In [94]:
# what kinds of uncounted people appear in subreddits? 
for person in uncounted: 
    for sr in people_counts_sr: 
        if person in people_counts_sr[sr]: 
            if people_counts_sr[sr][person] != 0: 
                print(sr, person, people_counts_sr[sr][person])

2016-03_foreveralone cougar 1
2016-03_mensrights cougar 5
2016-03_theredpill cougar 14
2016-03_thebluepill cougar 2
2016-03_asktrp cougar 4
2016-03_mgtow cougar 1
2016-03_seduction cougar 3
2016-03_askseddit cougar 4
2016-03_foreverunwanted cougar 2
2016-03_foreveralone try-hard 1
2016-03_theredpill try-hard 9
2016-03_thebluepill try-hard 2
2016-03_asktrp try-hard 8
2016-03_seduction try-hard 3
2016-03_askseddit try-hard 2
2016-03_marriedredpill try-hard 2
2016-03_mensrights keyboard jockey 1
2016-03_theredpill keyboard jockey 7
2016-03_asktrp keyboard jockey 1
2016-03_mensrights transtrender 2
2016-03_depression chang 1
2016-03_mensrights chang 1
2016-03_theredpill chang 1
2016-03_asktrp betafag 1
2016-03_mensrights ladies' man 1
2016-03_seduction ladies' man 1
2016-03_suicidewatch ladies' man 1
2016-03_mensrights kj 4
2016-03_theredpill kj 1
2016-03_theredpill volcel 1
2016-03_truecels volcel 1
2016-03_theredpill club girl 2
2016-03_depression void 95
2016-03_foreveralone void 9
2016

In [95]:
line2entities = defaultdict(list)
with open(LOGS + 'tagged_people/2016-03', 'r') as infile: 
    for i, line in enumerate(infile): 
        entities = line.split('\t')
        for entity in entities: 
            if entity.strip() == '': continue
            person = entity.lower().split()[2:]
            if person[0] == 'a' or person[0] == 'an' or person[0] == 'the': 
                person = person[1:]
            person = ' '.join(person)
            line2entities[i].append(person)

In [96]:
# Where are some words possibly being missed? 
i = 0
month = '2016-03'
with open(COMMENTS + 'RC_' + month + '/part-00000', 'r') as infile: 
    for line in infile: 
        d = json.loads(line)
        text = d['body']
        if 'omegas' in text: 
            if 'omegas' not in line2entities[i]: 
                print("------------------------------")
                print(text)
                print("------------------------------")
        i += 1
if os.path.exists(POSTS + 'RS_' + month + '/part-00000'): 
    post_path = POSTS + 'RS_' + month + '/part-00000'
else: 
    post_path = POSTS + 'RS_v2_' + month + '/part-00000'
with open(post_path, 'r') as infile: 
    for line in infile: 
        d = json.loads(line)
        text = d['selftext']
        if 'omegas' in text: 
            if 'omegas' not in line2entities[i]: 
                print("------------------------------")
                print(text)
                print("------------------------------")
        i += 1

------------------------------
&gt;The omegas are on a whole other level. Not only are they short, fat, and unattractive, these men are really invisible to you. Why? Because they do not even bother with women. They go play video games all day. They cannot for the life of them attract a woman. They go in front of a woman and stutter and twiddle their thumbs and do their best to muster “D-d-do y-you want, uh, um, to um l-like go on a d-d-d-date with me” and the poor woman cannot help but feel her vagina dry up like the Sahara Desert.

&gt;These men are about 80% of the male population to you (give or take 5%). 

Really? 80% of the male population is like that?

Dude, just because you're like that and your friends are like that doesn't mean the vast majority of the population is like that. 
------------------------------
------------------------------
So in 5 months you gained 17 pounds?  Ya.  Get your terminology straight.  That is straight up bulking.  Or are you one of them posers who 

------------------------------
Within the general ideas of Hypergamy, it's believed that woman only really go for the top 20% of men.  This would align with the 80/20 rule.  Whether this top 20% applies to looks or money or game, I don't know, but stay with me here.  

Would this mean that a guy just below the 80% threshold would get shit tested the most?  Woman are unlikely to test a guy who is DEFINITELY in the top tier, and they would likely ignore (not even bother to test) a guy who is DEFINITELY far away from that threshold.

But what about a guy who is 70-something % ??  Is he going to get the worst of it because it's not so obvious to her?  

Speaking for myself, I think I may have been there for a while and when my lifting got really consistant, I almost definitely surpassed the 80% mark.  Since then I get almost non-stop positive attention and no shit testing.  Meanwhile, my younger brother and some friends (who are betas bordering on omegas) never get any positive attention, 

In [ ]:
# What are possible new words not in glossaries? 
# note that right now, manual people does not include generic words
for person in all_entities.most_common(): 
    if person[0] not in people: 
        print(person)

('women', 33676)
('people', 24529)
('men', 20482)
('someone', 10379)
('man', 9416)
('woman', 8572)
('everyone', 6422)
('girl', 5592)
('girls', 5435)
('anyone', 5084)
('friends', 4762)
('no one', 3983)
('child', 3207)
('others', 3191)
('guy', 3153)
('other people', 2850)
('guys', 2699)
('kids', 2471)
('person', 2440)
('feminists', 2382)
('my parents', 2319)
('family', 2254)
('children', 2223)
('nobody', 2186)
('my friends', 1977)
('friend', 1817)
('my family', 1769)
('someone else', 1748)
('some people', 1702)
('most people', 1693)
('dude', 1637)
('my mom', 1579)
('', 1502)
('this guy', 1449)
('my dad', 1397)
('kid', 1382)
('this girl', 1349)
('my wife', 1330)
('my friend', 1199)
('boys', 1167)
('doctor', 1150)
('father', 1140)
('my mother', 1049)
('therapist', 1039)
('mother', 1011)
('wife', 989)
('male', 947)
('everyone else', 941)
('anyone else', 923)
('female', 857)
('your wife', 843)
('girlfriend', 813)
('somebody', 812)
('many people', 797)
('parents', 778)
('victim', 775)
('these

('boomers', 13)
('one group', 13)
('threat', 13)
('his opponent', 13)
('some loser', 13)
('beta boy', 13)
('african americans', 13)
('your sub', 13)
('all the guys', 13)
('guys like this', 13)
('magician', 13)
('co workers', 13)
('recluse', 13)
('that one', 13)
('hunters', 13)
('business owners', 13)
('only person who', 13)
('now', 13)
('subscribers', 13)
('hippie', 13)
('your roommates', 13)
('protesters', 13)
('most dudes', 13)
('their', 13)
('normal men', 13)
('pharmacist', 13)
('normal guys', 13)
('one gender', 13)
('my own parents', 13)
('my landlord', 13)
('people on this sub', 13)
('my ancestors', 13)
('groupies', 13)
('your ancestors', 13)
('bachelors', 13)
('enough women', 13)
('good ones', 13)
('sweetie', 13)
('typical woman', 13)
('fucking idiots', 13)
('some random person', 13)
('classmate', 13)
('your co - workers', 13)
('motherfuckers', 13)
('every single girl', 13)
('other man', 13)
('people of color', 13)
('up girls', 13)
('powerful men', 13)
('19 year old', 13)
('peopl

('publishers', 9)
('my attorney', 9)
('local', 9)
('any other man', 9)
('real alpha', 9)
('my younger sister', 9)
('that individual', 9)
('progressive', 9)
('young lady', 9)
('narrator', 9)
('hottest women', 9)
('it', 9)
('freelancer', 9)
('single girl', 9)
('man in', 9)
('western feminists', 9)
('all these guys', 9)
('creeper', 9)
('entertainers', 9)
('last', 9)
('old woman', 9)
('their captain', 9)
('traditional family', 9)
('beggars', 9)
('me', 9)
('sexual assault victim', 9)
('you wife', 9)
('good looking people', 9)
('my first boyfriend', 9)
('small dick', 9)
('their followers', 9)
('people in your life', 9)
('women there', 9)
('underage boys', 9)
('niggers', 9)
('some one else', 9)
('indian girls', 9)
('waiters', 9)
('fisherman', 9)
('outside world', 9)
('poor person', 9)
('misanthropic , ego - centric , arrogant , disrespectful , uncaring , asexual , distrustful , cynical asshole', 9)
('carpenters', 9)
('people close to you', 9)
('some students', 9)
('all other women', 9)
('some

('their teachers', 5)
('22 year old', 5)
('better fighter', 5)
('my oldest', 5)
('people with mental illness', 5)
('this particular woman', 5)
("women who do n't", 5)
('nihilists', 5)
('lucky one', 5)
('walking dead', 5)
('older ones', 5)
('some sucker', 5)
('economy', 5)
('each individual woman', 5)
('nice guy "', 5)
('one of my friends', 5)
('his students', 5)
('new dude', 5)
('women that', 5)
('vulnerable men', 5)
('sober person', 5)
('paedophile', 5)
('stemlord', 5)
('try hard weirdo', 5)
('people who are suicidal', 5)
('1 women', 5)
('her victims', 5)
('elderly women', 5)
('underage boy', 5)
('rps', 5)
('my dumbass', 5)
('evil bitches', 5)
('internet friend', 5)
('his siblings', 5)
('" friends "', 5)
('hi friend', 5)
('leader of men', 5)
('majority of guys', 5)
('some of those people', 5)
('second person', 5)
('muslim girl', 5)
('kiddies', 5)
('my high school class', 5)
('your subconscious', 5)
('spokesperson', 5)
('shrew', 5)
('his spouse', 5)
('skinny man', 5)
('nobodies', 5)
('

('virgin girl', 4)
('straight woman', 4)
('one female', 4)
('autistic people', 4)
('other sluts', 4)
('many of you', 4)
('your girl friend', 4)
('every student', 4)
('pt', 4)
('upper class women', 4)
('coworker of mine', 4)
('awkward person', 4)
('saint single mommy', 4)
('her beta orbiter', 4)
('what guy', 4)
('their company', 4)
('grown ass woman', 4)
('my current partner', 4)
('bank', 4)
('japanese men', 4)
('this one friend', 4)
('biological fathers', 4)
('no two people', 4)
("' friend", 4)
('good looking dude', 4)
('your soul mate', 4)
('accused student', 4)
('depressed man', 4)
('your soulmate', 4)
('long term partner', 4)
('our class', 4)
('social outcasts', 4)
('his community', 4)
('9/10', 4)
('faer', 4)
('decent wife', 4)
('some guy on the internet', 4)
('dozens', 4)
('your families', 4)
('rich white men', 4)
('paedophiles', 4)
('baristas', 4)
('some hot chick', 4)
('her older brother', 4)
('homeowner', 4)
('couple of women', 4)
('temp', 4)
('all white men', 4)
('angry guy', 4

...

('bottom 60 % of men', 1)
('men who know game', 1)
('modern lost boy', 1)
('shut in whose life is video games and porn before jacking off and going to bed only to repeat it the next day', 1)
('male pre - kindergarten teacher', 1)
('first girl you met', 1)
('certain artist', 1)
('everyone who makes music', 1)
('great reader', 1)
('plenty of siblings', 1)
('her room mates', 1)
('someone i knew blocking the', 1)
('of us on here', 1)
('people who deviate from thr norm that people', 1)
('last 5 girls', 1)
('woman like this walking the planet', 1)
('fucking minors', 1)
('" slags', 1)
('men coming from china', 1)
('more single , desperate chinese immigrants', 1)
('strange one', 1)
('average guy that got', 1)
('man with daddy issues', 1)
('ex athlete', 1)
('his first born son', 1)
('any man who had such a situation', 1)
('so many men who share a similar story', 1)
('lost boy', 1)
('some unattractive normal weight girls', 1)
('my " mom', 1)
('these folk', 1)
('really chill person', 1)
('manager

('people who have interesting lives', 1)
('presentators of that company', 1)
('couchsurfing.com', 1)
('regional burning man', 1)
('entry developers', 1)
('people who memorize it', 1)
('family that i have never gotten along with', 1)
('abusive step mother', 1)
('suicidally depressed mom', 1)
("gay or bi man who 's bored of other gays and are looking for a challenge to hook up with a straight guy", 1)
('burly austrian man with a inexplicably vast', 1)
('" real " men', 1)
('very stupid people', 1)
('all the club members', 1)
('one high quality woman', 1)
('disability administration', 1)
('hard working mom', 1)
('piece of shit wife beater', 1)
('disability people', 1)
('six year vet', 1)
('combat medic paratrooper', 1)
('non - criminals', 1)
('another brown person', 1)
('jack ass', 1)
('those below average guys', 1)
('hottest women on okc', 1)
('best - looking third', 1)
('plenty of potentially charming , even cute , girls', 1)
('absolute best - rated females', 1)
('hysterical manhating fe

('guy of her dreams', 1)
('some awesome guy who wanted to date / fuck her', 1)
('all the people you greet from the time you after you met them', 1)
('this chucklehead', 1)
('other organisations out there', 1)
('most ugle people', 1)
('number of women from various ethnic groups', 1)
("women from outside the ' west", 1)
("people i 've spoken two with similar dating experience", 1)
('standard whore', 1)
('fucking strangers', 1)
('someone i just met', 1)
('deadbeat who just wanted to take her for a ride and leech off her finances', 1)
('catch^1', 1)
('girl who tries to rope you in', 1)
('bad boys , has a modest n', 1)
('# 1 girlfriend', 1)
('unicorn who never dated bad boys , has a modest n - count and was always the # 1 girlfriend', 1)
('well - earning partner', 1)
('someone who has rejected you', 1)
('foreigner in a european country', 1)
('* rest * of us', 1)
('10/10 instagram model looking girl', 1)
('those university students', 1)
('total pushovers', 1)
('high - iq human being', 1)
('m

('women you talk to', 1)
('your wingman friend', 1)
('all new women', 1)
('all men in the digital age', 1)
("girl that did n't text me back until 1 month has passed", 1)
('people who actually enjoy it', 1)
('your sexy teenage daughter', 1)
('boston guy here', 1)
('disgusting slob', 1)
('all the assholes', 1)
('farm workers { people and/or beasts of burden } needed to operate the farm', 1)
("people who aren't working on that farm", 1)
('taxing entity', 1)
('some " bright fellow', 1)
('expected 100 people ( 50 %', 1)
('only 20 people', 1)
('mere 10 people', 1)
('trivial 1 %', 1)
('few ( or fewer ) people', 1)
('vast overwhelming majority of people', 1)
('really " smart " people', 1)
('so called " economists "', 1)
('even " nobel prize " winning " economists "', 1)
('condescending pricks', 1)
('some of the members', 1)
('many nice people', 1)
('some other user', 1)
('troubled girl', 1)
('type of person who was constantly " nagging , reminding , [ and ] correcting " your spouse', 1)
('5:00

('so many bigots', 1)
('traditional father', 1)
('all the no - fitness tard - liars on reddit', 1)
('joiner', 1)
('asses to other people without justification', 1)
('people who were jerks to me rather than myself', 1)
('your tormentors', 1)
('lot of girls out there who are going to share your same interests', 1)
('man hater', 1)
('" head "', 1)
('no single person', 1)
('each so', 1)
('current so', 1)
('bitter lonely person', 1)
('comp guys', 1)
('bunch of savage chimps', 1)
('type of person who is prepared to fight , claw and backstab their way to the top', 1)
('not one politician', 1)
('1200 guys', 1)
('someone who is emotionally manipulative and distant', 1)
('king lord', 1)
('many 9.0s', 1)
('driver test dude', 1)
('small minority group', 1)
('some male platonic friends', 1)
('must people here', 1)
('debra', 1)
('some guy living 1400 years ago', 1)
('secular people', 1)
('liberal secular women', 1)
('another group with a high fertility rate', 1)
('people with real social lives', 1)
